In [1]:
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import collections
import concurrent.futures
import datetime
import json
import termcolor
import re

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

In [2]:
local_models = True

if local_models:
    # Setup LLM
    from concordia.language_model import ollama_model
    model = ollama_model.OllamaLanguageModel(
        model_name='gemma',
        streaming=True
    )

    # Setup measurements and clock
    measurements = measurements_lib.Measurements()
    time_step = datetime.timedelta(minutes=20)
    SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

    START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
    clock = game_clock.MultiIntervalClock(
        start=SETUP_TIME,
        step_sizes=[time_step, datetime.timedelta(seconds=10)])

    # Setup sentence encoder
    from sentence_transformers import SentenceTransformer
    st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
    embedder = st5_model.encode

else:
    from concordia.language_model import gpt_model
    from concordia.language_model import gcloud_model
    # @title Language Model - pick your model and provide keys
    CLOUD_PROJECT_ID = '' #@param {type: 'string'}
    GPT_API_KEY = '' #@param {type: 'string'}
    GPT_MODEL_NAME = '' #@param {type: 'string'}

    USE_CLOUD = True #@param {type: 'boolean'}

    if USE_CLOUD:
        model = gcloud_model.CloudLanguageModel(project_id= CLOUD_PROJECT_ID)
    else:
        model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=GPT_MODEL_NAME)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()


In [3]:
# @title Generic memories are memories that all players and GM share.

shared_memories = [
    'Alice returns late from work and finds Dorothy at her doorstep.',
    "Dorothy has to move out of her house tonight and needs someone to help her move.",
]

# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(shared_memories)
    + '\n'
    + 'Summary:'
)

/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sure, here is a concise summary of the passage in a concise and insightful fashion:

Alice's late return from work finds her encountering Dorothy at her doorstep. Dorothy is forced to move out of her house that night and requires assistance with the move.

In [4]:
blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=shared_memories,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

In [5]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

scenario_premise = [

    (
        'Alice returns late from work and finds her friend Dorothy at her doorstep. '
        + 'Dorothy has to move out of her house tonight and needs someone to help her move.'
    ),
]
player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal=(
            "Dorothy has planned on Bob helping her move, but Bob didn't show up. "
        ),
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
]

In [6]:
mem = formative_memory_factory.make_memories(player_configs[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

## Alice: A Woman of Contradictions

**Job:** Graphic designer.

**Typical Day:** Alice is a creative force, working on projects that range from branding to website design. Her days are filled with brainstorming, sketching, and collaborating with clients.

**Goals:** To establish herself as a top designer, build a successful career, and leave a lasting impression on the world.

**Desires:** To travel the world, own a beautiful home, and experience happiness in her relationships.

**Hopes:** To see her designs implemented in prestigious projects and to inspire others.

**Dreams:** To create a legacy that transcends time and to leave the world a better place.

**Responsibilities:**

* Managing client relationships
* Meeting deadlines
* Maintaining creative integrity
* Balancing work and personal life

**Obligations:**

* Meeting professional standards
* Maintaining confidentiality
* Adherence to ethical guidelines

**Joy:**

* Creative freedom
* Appreciation of her work
* Collaboration w

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

**Formative Episode at Age 3:**

Three-year-old Alice, a creative force, was late from work one evening. As she arrived home, she found her neighbor, Dorothy, standing at her doorstep, distraught. Seeing her neighbor's distress, Alice felt compassion and offered her help.

**Formative Episode at Age 7:**

Seven-year-old Alice, a creative force, was late from work one evening. As she arrived home, she found her neighbor, Dorothy, standing at her doorstep, distraught. Seeing her neighbor's distress, Alice felt compassion and offered her help.

...

## Formative Episode at Age 21

Twenty-one-year-old Alice, a creative force, was late from work one evening. As she arrived home, she found her neighbor, Dorothy, standing at her doorstep, distraught. Seeing her neighbor's distress, Alice felt compassion and offered her help. Despite the challenges, Alice worked tirelessly to help Dorothy move out and provided her with a warm place to stay until she found a new home.

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [65]:
%autoreload 2
from examples.custom_components import tpb

behav = tpb.BehavioralChoices(
  "behav",
  model=model,
  memory=mem,
  player_config=player_configs[0],
  clock_now=None,
  num_memories_to_retrieve=100
)

behav.update()

## Alice's Potential Behaviours and Consequences

**Potential Behaviour:** Help Dorothy move out and provide her with a warm place to stay until she finds a new home.

**Positive Consequences:**

1. Dorothy is grateful for Alice's help and offers her a warm place to stay. (Value: 8, Likelihood: 80%)
2. Alice feels good knowing she helped someone in need. (Value: 7, Likelihood: 70%)
3. Alice feels accomplished and satisfied with her ability to help others. (Value: 9, Likelihood: 60%)

**Negative Consequences:**

1. Alice is exhausted from work and has to help her grandmother all day the next day. (Value: -2, Likelihood: 40%)
2. Alice is late for work the next day due to her extended absence. (Value: -4, Likelihood: 30%)
3. Alice feels overwhelmed and stressed by the situation. (Value: -3, Likelihood: 20%)

**Potential Behaviour:** Not help Dorothy move out.

**Positive Consequences:**

1. Alice is able to rest and recover from work. (Value: 6, Likelihood: 90%)
2. Alice has more time to 

In [69]:
behav.jsonify()

- 1. Dorothy is grateful for Alice's help and offers her a warm place to stay. (Value: 8, Likelihood: 80%) -
- 2. Alice feels good knowing she helped someone in need. (Value: 7, Likelihood: 70%) -
- 3. Alice feels accomplished and satisfied with her ability to help others. (Value: 9, Likelihood: 60%) -
- 1. Alice is exhausted from work and has to help her grandmother all day the next day. (Value: -2, Likelihood: 40%) -
- 2. Alice is late for work the next day due to her extended absence. (Value: -4, Likelihood: 30%) -
- 3. Alice feels overwhelmed and stressed by the situation. (Value: -3, Likelihood: 20%) -
- 1. Alice is able to rest and recover from work. (Value: 6, Likelihood: 90%) -
- 2. Alice has more time to herself and can pursue her own interests. (Value: 5, Likelihood: 80%) -
- 3. Alice feels relaxed and refreshed. (Value: 7, Likelihood: 70%) -
- 1. Dorothy is upset and feels abandoned. (Value: -4, Likelihood: 50%) -
- 2. Dorothy has to find alternative arrangements for moving 

[{'behaviour': 'Help Dorothy move out and provide her with a warm place to stay until she finds a new home.',
  'consequences': [{'description': "Dorothy is grateful for Alice's help and offers her a warm place to stay.",
    'value': 8,
    'likelihood': 0.8},
   {'description': 'Alice feels good knowing she helped someone in need.',
    'value': 7,
    'likelihood': 0.7},
   {'description': 'Alice feels accomplished and satisfied with her ability to help others.',
    'value': 9,
    'likelihood': 0.6},
   {'description': 'Alice is exhausted from work and has to help her grandmother all day the next day.',
    'value': -2,
    'likelihood': 0.4},
   {'description': 'Alice is late for work the next day due to her extended absence.',
    'value': -4,
    'likelihood': 0.3},
   {'description': 'Alice feels overwhelmed and stressed by the situation.',
    'value': -3,
    'likelihood': 0.2}]},
 {'behaviour': 'Not help Dorothy move out.',
  'consequences': [{'description': 'Alice is able 

In [70]:
%autoreload 2
from examples.custom_components import tpb

people = tpb.SubjectiveNorms(
  "people",
  model=model,
  memory=mem,
  player_config=player_configs[0],
  components=[behav],
  clock_now=None,
  num_memories_to_retrieve=100
)

people.update()

- 1. Dorothy is grateful for Alice's help and offers her a warm place to stay. (Value: 8, Likelihood: 80%) -
- 2. Alice feels good knowing she helped someone in need. (Value: 7, Likelihood: 70%) -
- 3. Alice feels accomplished and satisfied with her ability to help others. (Value: 9, Likelihood: 60%) -
- 1. Alice is exhausted from work and has to help her grandmother all day the next day. (Value: -2, Likelihood: 40%) -
- 2. Alice is late for work the next day due to her extended absence. (Value: -4, Likelihood: 30%) -
- 3. Alice feels overwhelmed and stressed by the situation. (Value: -3, Likelihood: 20%) -
- 1. Alice is able to rest and recover from work. (Value: 6, Likelihood: 90%) -
- 2. Alice has more time to herself and can pursue her own interests. (Value: 5, Likelihood: 80%) -
- 3. Alice feels relaxed and refreshed. (Value: 7, Likelihood: 70%) -
- 1. Dorothy is upset and feels abandoned. (Value: -4, Likelihood: 50%) -
- 2. Dorothy has to find alternative arrangements for moving 

In [111]:
print(people.jsonify())

[{'behaviour': 'Help Dorothy move out and provide her with a warm place to stay until she finds a new home.', 'consequences': [{'description': "Dorothy is grateful for Alice's help and offers her a warm place to stay.", 'value': 8, 'likelihood': 0.8}, {'description': 'Alice feels good knowing she helped someone in need.', 'value': 7, 'likelihood': 0.7}, {'description': 'Alice feels accomplished and satisfied with her ability to help others.', 'value': 9, 'likelihood': 0.6}, {'description': 'Alice is exhausted from work and has to help her grandmother all day the next day.', 'value': -2, 'likelihood': 0.4}, {'description': 'Alice is late for work the next day due to her extended absence.', 'value': -4, 'likelihood': 0.3}, {'description': 'Alice feels overwhelmed and stressed by the situation.', 'value': -3, 'likelihood': 0.2}], 'people': [{'person': 'Grandmother', 'approval': 8}, {'person': 'Friend', 'approval': 6}, {'person': 'Neighbor', 'approval': 2}, {'person': 'Teacher', 'approval'

In [113]:
%autoreload 2
from examples.custom_components import tpb

motivs = tpb.PersonMotivations(
  "motivs",
  model=model,
  memory=mem,
  player_config=player_configs[0],
  components=[people],
  clock_now=None
)

motivs.update()

**Grandmother:** (Motivation: 20)
Alice is motivated to take into consideration her grandmother's approval, but not as much as other people. She has a lot of other commitments and is exhausted after work.

**Friend:** (Motivation: 30)
Alice is more motivated to take into consideration her friend's approval than her grandmother's. She would like to maintain a positive relationship with her friend.

**Neighbor:** (Motivation: 60)
Alice is highly motivated to take into consideration her neighbor's approval. She has a close relationship with her neighbor and feels compassion for her distress.

**Teacher:** (Motivation: 10)
Alice is not very motivated to take into consideration her teacher's approval. She is not particularly close to her teacher and does not have a lot of interaction with them.

**Colleague:** (Motivation: 25)
Alice is motivated to take into consideration her colleague's approval to some extent, but not as much as her neighbor or friend. She would like to maintain a positiv

In [128]:
json_test = motivs.jsonify()

In [130]:
import json
json_test

[{'behaviour': 'Help Dorothy move out and provide her with a warm place to stay until she finds a new home.',
  'consequences': [{'description': "Dorothy is grateful for Alice's help and offers her a warm place to stay.",
    'value': 8,
    'likelihood': 0.8},
   {'description': 'Alice feels good knowing she helped someone in need.',
    'value': 7,
    'likelihood': 0.7},
   {'description': 'Alice feels accomplished and satisfied with her ability to help others.',
    'value': 9,
    'likelihood': 0.6},
   {'description': 'Alice is exhausted from work and has to help her grandmother all day the next day.',
    'value': -2,
    'likelihood': 0.4},
   {'description': 'Alice is late for work the next day due to her extended absence.',
    'value': -4,
    'likelihood': 0.3},
   {'description': 'Alice feels overwhelmed and stressed by the situation.',
    'value': -3,
    'likelihood': 0.2}],
  'people': [{'person': 'Grandmother', 'approval': 8, 'motivation': 20},
   {'person': 'Friend',